In [ ]:
# If this fails `pip install requests` in terminal
# If that fails http://docs.python-requests.org/en/latest/user/install/#install

import requests
import json

morph_api_url = 'https://api.morph.io/okfn/opentrials-clinicaltrailsgov-data/data.json'

# REPLACE THIS WITH THE GITHUB TOKEN FOR MORPH.IO
morph_api_key = 'qRunqlA1Fu9lJs3ApuK7'

# REPLACE THIS WITH YOUR JAMDB INFO
jam_base_url = 'http://localhost:1212/v1/'
namespace = 'OPENTRIALS'
collection = 'data'
username='scrapi'
password='password'


jam_data_url = '{}namespaces/{}/collections/{}/documents'.format(jam_base_url, namespace, collection)
jam_auth_url = '{}auth/'.format(jam_base_url)
jam_auth_payload = {
    'data': {
        'type':'users',
        'attributes': {
            'provider':'self',
            'collection': 'users',
            'namespace': namespace,
            'username': username,
            'password': password
        }
    }
}

# REPLACE THIS WITH THE PRIMARY KEY IN THE MORPH.IO DATA
id_key_name = 'nct_id'

# Results per page
limit = 2000

# start on this result
offset = 0

error = False
total = 0
first = True

current_token = None

def get_auth_token():
    print('Getting new jam auth token')
    r = requests.post(jam_auth_url, json=jam_auth_payload)
    if r.ok:
        print('Got new jam auth token')
        return r.json()['data']['attributes']['token']
    else:
        print(r.json())

    
def create_collection(auth_token):
    r = requests.post('{}namespaces/{}/collections'.format(jam_base_url, namespace,), 
                     json=dict(
                        data=dict(
                            type='collections',
                            id=collection,
                            attributes=dict()
                        )
                    ),
                    headers=dict(Authorization=auth_token)
                )
    if r.ok:
        print('Created collection `data`')
    elif r.status_code == 409:
        print('Collection exists')
    else:
        print('Error creating collection!')
        print(r.json())
            
            
def json_loads_keys(data):
    loaded_json_data = {}
    for key, value in data.items():
        try:
            json_value = json.loads(value)
            loaded_json_data[key] = json_value
        except (json.JSONDecodeError, TypeError):
            loaded_json_data[key] = value
    return loaded_json_data
    

collection_created = False
while not error:
    auth_token = get_auth_token()
    if not collection_created:
        create_collection(auth_token)
        collection_created = True
        
    # get {limit} records from morph.io
    r = requests.get(morph_api_url, params={
      'key': morph_api_key,
      'query': 'select * from "data" limit {} offset {}'.format(limit, offset)
    })
    if not r.ok:
        # morph.io response was an error
        error = True
        print('Got Error!')
        print(r.json())
    else:
        # morph.io response was success
        count = len(r.json())

        if count == 0:
            # we didn't get any results
            error = True
            print('Done.')
        else:
            # we did get results
            total = total + count
            print('Got {} things for a total of {}.'.format(count, total))

            temp_data = r.json()
            # loop through the items from the response
            for item in temp_data:
                loaded_item = json_loads_keys(item)
#                 with open('./{}.json'.format(loaded_item[id_key_name]), 'w') as f:
#                     f.write(json.dumps(loaded_item, indent=4))
#                     f.close()
                # create the json-api compatible jamdb payload
                data_dict = dict(data=dict(
                                            type='documents',
                                            id=loaded_item[id_key_name],
                                            attributes=loaded_item)
                                        )
                # make jam request
                jam_r = requests.post(jam_data_url,
                                      json=data_dict,
                                      headers=dict(Authorization=auth_token))
                if jam_r.ok:
                    continue
                elif jam_r.status_code == 409:
                    print('Key {} Exists!'.format(loaded_item[id_key_name]))
                else:
                    # response not ok, print result to see error message
                    print(jam_r.content)

            # increase offset, reasons
            offset = offset + limit

Getting new jam auth token
Got new jam auth token
Created collection `data`
Got 2000 things for a total of 2000.
Key NCT00265694 Exists!
Key NCT00260767 Exists!
Key NCT00000111 Exists!
Key NCT00257296 Exists!
Key NCT00254111 Exists!
Key NCT00000179 Exists!
Key NCT00000201 Exists!
Key NCT00259168 Exists!
Key NCT00244127 Exists!
Key NCT00242177 Exists!
Key NCT00000266 Exists!
Key NCT00243360 Exists!
Key NCT00226850 Exists!
Key NCT00000323 Exists!
Key NCT00225329 Exists!
Key NCT00223535 Exists!
Key NCT00224393 Exists!
Key NCT00000474 Exists!
Key NCT00214708 Exists!
Key NCT00000526 Exists!
Key NCT00000543 Exists!
Key NCT00000553 Exists!
Key NCT00217178 Exists!
Key NCT00000759 Exists!
Key NCT00200499 Exists!
Key NCT00197899 Exists!
Key NCT00002012 Exists!
Key NCT00002013 Exists!
Key NCT00002022 Exists!
Key NCT00198926 Exists!
Key NCT00002037 Exists!
Key NCT00002043 Exists!
Key NCT00196287 Exists!
Key NCT00196703 Exists!
Key NCT00197483 Exists!
Key NCT00002093 Exists!
Key NCT00002097 Exists!